**목적**
- 어떤 고객이 탈퇴할지 예측하여, 이를 방지하기 위한 캠페인을 실행하자

**정보**
- 3, 4장에서 가공된 데이터 사용
- 데이터 설명
    1. use_log_months.csv
        - 회원의 스포츠센터 이용 이력을 연월/고객별로 집계한 데이터
    2. customer_join.csv
        - 이용 이력을 포함한 고객 데이터

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Data-Load" data-toc-modified-id="Data-Load-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Data Load</a></span></li><li><span><a href="#데이터-전처리" data-toc-modified-id="데이터-전처리-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>데이터 전처리</a></span><ul class="toc-item"><li><span><a href="#월별-해당월-+-1개월-전-data" data-toc-modified-id="월별-해당월-+-1개월-전-data-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>월별 해당월 + 1개월 전 data</a></span></li><li><span><a href="#탈퇴-회원---탈퇴-전월의-이용-데이터" data-toc-modified-id="탈퇴-회원---탈퇴-전월의-이용-데이터-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>탈퇴 회원 - 탈퇴 전월의 이용 데이터</a></span></li><li><span><a href="#지속-회원-데이터" data-toc-modified-id="지속-회원-데이터-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>지속 회원 데이터</a></span></li><li><span><a href="#탈퇴-회원-+-지속-회원" data-toc-modified-id="탈퇴-회원-+-지속-회원-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>탈퇴 회원 + 지속 회원</a></span></li><li><span><a href="#변수-추가---회원-기간" data-toc-modified-id="변수-추가---회원-기간-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>변수 추가 - 회원 기간</a></span></li><li><span><a href="#결측치-제거" data-toc-modified-id="결측치-제거-2.6"><span class="toc-item-num">2.6&nbsp;&nbsp;</span>결측치 제거</a></span></li><li><span><a href="#필요-변수만-선택" data-toc-modified-id="필요-변수만-선택-2.7"><span class="toc-item-num">2.7&nbsp;&nbsp;</span>필요 변수만 선택</a></span></li><li><span><a href="#더미-변수-생성" data-toc-modified-id="더미-변수-생성-2.8"><span class="toc-item-num">2.8&nbsp;&nbsp;</span>더미 변수 생성</a></span></li></ul></li><li><span><a href="#탈퇴-예측" data-toc-modified-id="탈퇴-예측-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>탈퇴 예측</a></span><ul class="toc-item"><li><span><a href="#Decision-Tree-모델링-및-평가" data-toc-modified-id="Decision-Tree-모델링-및-평가-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Decision Tree 모델링 및 평가</a></span></li><li><span><a href="#하이퍼파라미터-튜닝" data-toc-modified-id="하이퍼파라미터-튜닝-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>하이퍼파라미터 튜닝</a></span></li><li><span><a href="#변수-중요도" data-toc-modified-id="변수-중요도-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>변수 중요도</a></span></li><li><span><a href="#회원-탈퇴-예측" data-toc-modified-id="회원-탈퇴-예측-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>회원 탈퇴 예측</a></span></li></ul></li></ul></div>

In [1]:
import pandas as pd
import numpy as np

from dateutil.relativedelta import relativedelta

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

# 그래프 한글 폰트
from matplotlib import font_manager, rc
font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
rc('font', family=font_name)

# 그래프 마이너스 폰트 깨지는 문제 대처
import matplotlib as mpl
mpl.rcParams['axes.unicode_minus'] = False

import warnings
warnings.filterwarnings('ignore')

# Data Load

In [2]:
customer = pd.read_csv('data_code/5장/customer_join.csv')
customer.head()

,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period
0,OA832399,XXXX,C01,F,2015-05-01,NaN,CA1,0,종일,10500,일반,4.833333,5.0,8,2,1,2019-04-30,47
1,PL270116,XXXXX,C01,M,2015-05-01,NaN,CA1,0,종일,10500,일반,5.083333,5.0,7,3,1,2019-04-30,47
2,OA974876,XXXXX,C01,M,2015-05-01,NaN,CA1,0,종일,10500,일반,4.583333,5.0,6,3,1,2019-04-30,47
3,HD024127,XXXXX,C01,F,2015-05-01,NaN,CA1,0,종일,10500,일반,4.833333,4.5,7,2,1,2019-04-30,47
4,HD661448,XXXXX,C03,F,2015-05-01,NaN,CA1,0,야간,6000,일반,3.916667,4.0,6,1,1,2019-04-30,47


In [3]:
uselog_months = pd.read_csv('data_code/5장/use_log_months.csv')
uselog_months.head()

,연월,customer_id,count
0,201804,AS002855,4
1,201804,AS009013,2
2,201804,AS009373,3
3,201804,AS015315,6
4,201804,AS015739,7


# 데이터 전처리

## 월별 해당월 + 1개월 전 data
- 몇 개월 만에 그만둔 회원이 많아, 활용 데이터 기간을 짧게 잡기 

In [4]:
year_months = list(uselog_months['연월'].unique())
uselog = pd.DataFrame()

for i in range(1, len(year_months)):
    tmp = uselog_months[uselog_months['연월'] == year_months[i]].copy()
    tmp.rename(columns={'count':'count_0'}, inplace=True)
    
    tmp_before = uselog_months[uselog_months['연월'] == year_months[i-1]][['customer_id','count']].copy()
    tmp_before.rename(columns={'count':'count_1'}, inplace=True)
    
    tmp = pd.merge(tmp, tmp_before, on='customer_id', how='left')
    uselog = pd.concat([uselog, tmp], ignore_index=True)
uselog.head()

,연월,customer_id,count_0,count_1
0,201805,AS002855,5,4.0
1,201805,AS009373,4,3.0
2,201805,AS015233,7,NaN
3,201805,AS015315,3,6.0
4,201805,AS015739,5,7.0


## 탈퇴 회원 - 탈퇴 전월의 이용 데이터
- 해당 스포츠 센터는 탈퇴 신청한 달의 다음 달에 탈퇴 처리가 됨 (end_date의 전 달에 탈퇴 신청함)
- 따라서, end_date의 1개월 전 월 기준으로 uselog['count_0','count_1'] 결합
    - ex. end_date 9월 -> 7월 data로 8월에 탈퇴 신청할 확률 예측

In [5]:
# end_date - 1months
exit_customer = customer[customer['is_deleted'] == 1].copy()
exit_customer['exit_date'] = None
exit_customer['end_date'] = pd.to_datetime(exit_customer['end_date'])

for i in range(len(exit_customer)):
    exit_customer['exit_date'].iloc[i] = exit_customer['end_date'].iloc[i] - relativedelta(months=1)

exit_customer['연월'] = pd.to_datetime(exit_customer['exit_date']).dt.strftime('%Y%m')
exit_customer['연월'] = exit_customer['연월'].astype(str)
uselog['연월'] = uselog['연월'].astype(str)
exit_uselog = pd.merge(uselog, exit_customer, on=['customer_id','연월'], how='left')
print(exit_uselog.shape)
exit_uselog.head()

(33851, 22)


,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
0,201805,AS002855,5,4.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,201805,AS009373,4,3.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,201805,AS015233,7,NaN,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,201805,AS015315,3,6.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,201805,AS015739,5,7.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# 탈퇴한 회원의 탈퇴 전월 data만 남기기 (결측치 제거)
exit_uselog = exit_uselog.dropna(subset=['name'])
print(len(exit_uselog))
print(len(exit_uselog['customer_id'].unique()))
exit_uselog.head()

1104
1104


,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
19,201805,AS055680,3,3.0,XXXXX,C01,M,2018-03-01,2018-06-30,CA1,...,10500.0,일반,3.000000,3.0,3.0,3.0,0.0,2018-06-30,3.0,2018-05-30 00:00:00
57,201805,AS169823,2,3.0,XX,C01,M,2017-11-01,2018-06-30,CA1,...,10500.0,일반,3.000000,3.0,4.0,2.0,1.0,2018-06-30,7.0,2018-05-30 00:00:00
110,201805,AS305860,5,3.0,XXXX,C01,M,2017-06-01,2018-06-30,CA1,...,10500.0,일반,3.333333,3.0,5.0,2.0,0.0,2018-06-30,12.0,2018-05-30 00:00:00
128,201805,AS363699,5,3.0,XXXXX,C01,M,2018-02-01,2018-06-30,CA1,...,10500.0,일반,3.333333,3.0,5.0,2.0,0.0,2018-06-30,4.0,2018-05-30 00:00:00
147,201805,AS417696,1,4.0,XX,C03,F,2017-09-01,2018-06-30,CA1,...,6000.0,일반,2.000000,1.0,4.0,1.0,0.0,2018-06-30,9.0,2018-05-30 00:00:00


## 지속 회원 데이터
- 지속 회원 데이터만!

In [7]:
conti_customer = customer[customer['is_deleted']==0].copy()
conti_uselog = pd.merge(uselog, conti_customer, on='customer_id', how='left')
print(len(conti_uselog))
conti_uselog.dropna(subset=['name'], inplace=True)
print(len(conti_uselog))

33851
27422


- 이탈 data 1,104개, 지속 data 27,422개 => 불균형 data
- 따라서 지속 회원의 샘플 수 조정 필요

In [8]:
# data를 랜덤으로 섞은 후, customer_id 기준으로 중복제거 => 각 id당 1개의 row(data)만 남도록  = 이탈 data와 동일
# frac: 전체 row에서 몇%의 데이터를 return할 것인지 -> 1 = 100% = 모든 데이터 반환
conti_uselog = conti_uselog.sample(frac=1).reset_index(drop=True)   
conti_uselog.drop_duplicates(subset=['customer_id'], inplace=True)
conti_uselog.shape

(2842, 21)

- data가 2,842개로 줄어들어, 이탈 데이터와의 차이가 많이 줄어듬

## 탈퇴 회원 + 지속 회원 
- 세로 결합

In [9]:
predict_data = pd.concat([conti_uselog, exit_uselog], ignore_index=True)
print(predict_data.shape)
predict_data.head()

(3946, 22)


,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
0,201901,HI305198,8,9.0,XXXXXX,C02,F,2018-10-11,NaN,CA1,...,7500.0,일반,9.000000,9.0,10.0,8.0,1.0,2019-04-30,6.0,NaN
1,201903,HI693441,7,5.0,XXXX,C03,M,2017-07-01,NaN,CA2,...,6000.0,입회비반액할인,6.333333,6.5,10.0,3.0,1.0,2019-04-30,21.0,NaN
2,201809,IK844882,10,9.0,XXX,C01,F,2017-09-01,NaN,CA1,...,10500.0,일반,7.583333,7.5,10.0,5.0,1.0,2019-04-30,19.0,NaN
3,201812,TS150659,5,6.0,XX,C03,F,2017-06-01,NaN,CA1,...,6000.0,일반,4.833333,5.0,9.0,1.0,1.0,2019-04-30,22.0,NaN
4,201812,HI900494,5,5.0,XX,C03,M,2015-06-01,NaN,CA1,...,6000.0,일반,5.750000,6.0,8.0,3.0,1.0,2019-04-30,46.0,NaN


## 변수 추가 - 회원 기간
- 앞서 3, 4장에서 고객 행동 분석 시 회원 기간이 고객의 특성을 나타내는 변수 중 하나임을 확인했음

In [10]:
predict_data['period'] = 0
predict_data['now_date'] = pd.to_datetime(predict_data['연월'], format='%Y%m')
predict_data['start_date'] = pd.to_datetime(predict_data['start_date'])

# diff months
for i in range(len(predict_data)):
    delta = relativedelta(predict_data['now_date'][i], predict_data['start_date'][i])
    predict_data['period'][i] = int(12 * delta.years + delta.months)
predict_data.head()

,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date,period,now_date
0,201901,HI305198,8,9.0,XXXXXX,C02,F,2018-10-11,NaN,CA1,...,9.000000,9.0,10.0,8.0,1.0,2019-04-30,6.0,NaN,2,2019-01-01
1,201903,HI693441,7,5.0,XXXX,C03,M,2017-07-01,NaN,CA2,...,6.333333,6.5,10.0,3.0,1.0,2019-04-30,21.0,NaN,20,2019-03-01
2,201809,IK844882,10,9.0,XXX,C01,F,2017-09-01,NaN,CA1,...,7.583333,7.5,10.0,5.0,1.0,2019-04-30,19.0,NaN,12,2018-09-01
3,201812,TS150659,5,6.0,XX,C03,F,2017-06-01,NaN,CA1,...,4.833333,5.0,9.0,1.0,1.0,2019-04-30,22.0,NaN,18,2018-12-01
4,201812,HI900494,5,5.0,XX,C03,M,2015-06-01,NaN,CA1,...,5.750000,6.0,8.0,3.0,1.0,2019-04-30,46.0,NaN,42,2018-12-01


## 결측치 제거
- 머신러닝 모델링 전, 결측치는 제거하거나 다른 값으로 대체해줘야 함

In [11]:
predict_data.isnull().sum()

연월                      0
customer_id             0
count_0                 0
count_1               257
name                    0
class                   0
gender                  0
start_date              0
end_date             2842
campaign_id             0
is_deleted              0
class_name              0
price                   0
campaign_name           0
mean                    0
median                  0
max                     0
min                     0
routine_flg             0
calc_date               0
membership_period       0
exit_date            2842
period                  0
now_date                0
dtype: int64

- end_date, exit_date는 탈퇴 회원만 정보를 가지고 있어, 결측치 = 지속 회원임을 의미
- count_1의 결측치가 있으면 의미가 없으므로 해당 컬럼에 결측치가 있으면 row 제거

In [12]:
# count_1 기준 data 제거
predict_data.dropna(subset=['count_1'], inplace=True)
predict_data.isnull().sum()

연월                      0
customer_id             0
count_0                 0
count_1                 0
name                    0
class                   0
gender                  0
start_date              0
end_date             2637
campaign_id             0
is_deleted              0
class_name              0
price                   0
campaign_name           0
mean                    0
median                  0
max                     0
min                     0
routine_flg             0
calc_date               0
membership_period       0
exit_date            2637
period                  0
now_date                0
dtype: int64

## 필요 변수만 선택

In [13]:
# 분류 예측에 사용할 변수 선택, 목적 변수
target_col = ['count_1','period','campaign_name','class_name','gender','routine_flg','is_deleted']
predict_data = predict_data[target_col]
predict_data.head()

,count_1,period,campaign_name,class_name,gender,routine_flg,is_deleted
0,9.0,2,일반,주간,F,1.0,0.0
1,5.0,20,입회비반액할인,야간,M,1.0,0.0
2,9.0,12,일반,종일,F,1.0,0.0
3,6.0,18,일반,야간,F,1.0,0.0
4,5.0,42,일반,야간,M,1.0,0.0


## 더미 변수 생성
- 카테고리 변수를 더미화 시키기
- 변수 속성 개수 n개 -> n-1개 더미 변수 생성

In [14]:
predict_data = pd.get_dummies(predict_data, drop_first=True)  # drop_first=True : n-1
predict_data.head()

,count_1,period,routine_flg,is_deleted,campaign_name_입회비무료,campaign_name_입회비반액할인,class_name_종일,class_name_주간,gender_M
0,9.0,2,1.0,0.0,0,0,0,1,0
1,5.0,20,1.0,0.0,0,1,0,0,1
2,9.0,12,1.0,0.0,0,0,1,0,0
3,6.0,18,1.0,0.0,0,0,0,0,0
4,5.0,42,1.0,0.0,0,0,0,0,1


# 탈퇴 예측

## Decision Tree 모델링 및 평가
- 직관적이며 이해하기 쉬움 -> 모델 설명 가능, 처음에 test용으로 자주 사용됨 

In [15]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

exit = predict_data[predict_data['is_deleted']==1]
conti = predict_data[predict_data['is_deleted']==0].sample(len(exit))    # 탈퇴 회원 샘플수(1104)에 맞추기

X = pd.concat([exit, conti], ignore_index=True)  # 다시 결함
y = X['is_deleted']
del X['is_deleted']

X_train, X_test, y_train, y_test = train_test_split(X,y)

model = DecisionTreeClassifier(random_state=0)
model.fit(X_train, y_train)
y_test_pred = model.predict(X_test)
y_test_pred

array([1., 1., 1., 1., 1., 0., 1., 1., 0., 0., 0., 0., 1., 1., 1., 1., 1.,
       0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 1., 1., 0., 1., 1., 1., 0.,
       0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 1., 1.,
       0., 1., 1., 0., 1., 1., 0., 0., 1., 1., 1., 0., 0., 0., 0., 1., 0.,
       0., 1., 0., 0., 0., 1., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1.,
       0., 0., 0., 1., 1., 0., 1., 1., 1., 0., 0., 1., 1., 1., 0., 1., 0.,
       0., 0., 1., 1., 0., 0., 1., 0., 0., 1., 0., 1., 0., 1., 1., 1., 1.,
       1., 0., 1., 1., 1., 0., 1., 0., 0., 0., 0., 1., 0., 1., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 1., 1., 0., 1.,
       1., 1., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 1., 0.,
       1., 1., 0., 1., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0.,
       1., 0., 1., 1., 0., 1., 1., 0., 1., 1., 0., 0., 1., 0., 1., 1., 1.,
       1., 0., 1., 1., 1., 1., 0., 0., 1., 1., 1., 0., 0., 1., 1., 1., 1.,
       1., 0., 1., 0., 0.

In [16]:
print(model.score(X_train, y_train))
print(model.score(X_test, y_test))

0.9740177439797212
0.876425855513308


- 학습용 데이터를 이용한 예측 정확도는 97%, 평가용 데이터를 이용한 예측 정확도 88%
- 학습용 데이터에 너무 오퍼피팅됨(과적합)
    - 데이터 늘리기, 변수 재검토, 모델 파라미터 변경 등 방법 적용하며 좀 더 이상적인(두 점수간 차이가 적은) 모델 만들어 나가기!!

## 하이퍼파라미터 튜닝

In [17]:
# 트리 깊이 얕게 해서 모델 단순화
model = DecisionTreeClassifier(random_state=0, max_depth=5)
model.fit(X_train, y_train)

print(model.score(X_train, y_train))
print(model.score(X_test, y_test))

0.9296577946768061
0.908745247148289


- 학습 데이터 성능 낮아지고(오버피팅 완화), 평가 데이터 성능 높아지며 둘 점수 간의 차이가 작아짐! -> 일반화 성능 up

## 변수 중요도
- 모델에 기여하는 정도 (중요도)

In [18]:
importance = pd.DataFrame({'feature_names':X.columns, 'coefficient':model.feature_importances_})
importance.sort_values(by='coefficient', ascending=False).style.background_gradient()

,feature_names,coefficient
1,period,0.512107
0,count_1,0.354954
2,routine_flg,0.129324
4,campaign_name_입회비반액할인,0.003615
3,campaign_name_입회비무료,0.000000
5,class_name_종일,0.000000
6,class_name_주간,0.000000
7,gender_M,0.000000


- 회원기간, 1개월 전의 이용 횟수, 정기 이용 여부 순으로 모델에 기여하고 있음

##  회원 탈퇴 예측
- sample test data 예측

In [19]:
X.head(2)

,count_1,period,routine_flg,campaign_name_입회비무료,campaign_name_입회비반액할인,class_name_종일,class_name_주간,gender_M
0,3.0,2,0.0,0,0,1,0,1
1,3.0,6,1.0,0,0,1,0,1


In [20]:
# sample test
count_1 = 3
routine_flg = 1
period = 10
campaign_name = '입회비무료'
class_name = '종일'
gender = 'M'

In [21]:
# input data processing

# 카테고리형 변수 더미 처리
if campaign_name == '입회비반액할인': # campaign_name
    campaign_name_list = [0,1]
elif campaign_name == '입회비무료':
    campaign_name_list = [1,0]
elif campaign_name == '일반':
    campaign_name_list = [0,0]
if class_name == '종일':            # class
    class_name_list = [1,0]
elif class_name == '주간':
    class_name_list = [0,1]
elif class_name == '야간':
    class_name_list = [0,0]
if gender == 'F':
    gender_list = [0]
elif gender == 'M':
    gender_list = [1]
# 수치형
input_data = [count_1,period,routine_flg]
# 더미 변수 추가
input_data.extend(campaign_name_list)
input_data.extend(class_name_list)
input_data.extend(gender_list)

In [22]:
input_data

[3, 10, 1, 1, 0, 1, 0, 1]

In [23]:
print(model.predict([input_data]))    # 예측 class
print(model.predict_proba([input_data]))   # 예측 확률

[1.]
[[0.00268097 0.99731903]]


- 탈퇴가 99.7% 확률로 예측됨